In [6]:
from pybit.unified_trading import HTTP
import numpy as np
import time
import torch

In [2]:
SYMBOL = 'SOLUSDT'
API_KEY = 'Z8QsHgyTpwinr9K2Ya'
API_SECRET = '5QfjbglBfFK5A8RqGvcvdw8hCF4Yjv5sByOn'
cl = HTTP(
    
)

In [3]:
START_TIMESTAMP = '1711929617184'
time_series = 1000 #  количество изначальных временных рядов в разные промежутки времени, с количеством измерений t_s_l
time_series_length = 20

time_stamps = [START_TIMESTAMP]

train_data = np.ndarray(shape=(time_series, time_series_length))
volumes = np.ndarray(shape=time_series_length*time_series)
all_price = np.ndarray(shape=time_series_length*time_series)

for i in range(time_series):
    kline = cl.get_kline(
        category="spot",
        symbol=SYMBOL,
        start=time_stamps[-1],
        interval=1,
        limit = time_series_length
    )
    if i % 115 == 0:
        time.sleep(5)
    time_stamps.append(str( int(kline.get('result', []).get('list', [])[0][0]) + 10000))
    
    volumes[i*time_series_length:(i+1)*time_series_length] = np.array([i[5] for i in kline.get('result', []).get('list', [])[::-1]], dtype=float)
    all_price[i*time_series_length:(i+1)*time_series_length] = np.array([i[4] for i in kline.get('result', []).get('list', [])[::-1]], dtype=float)
    
    prices = np.array([i[4] for i in kline.get('result', []).get('list', [])[::-1]], dtype=float)         
    train_data[i] = np.array(prices, dtype=float)

In [51]:
train_size = int(time_series * 0.8)
prediction_length = 10
idxs = np.random.choice((time_series-1), size=train_size)
x_train, y_train, = torch.tensor(train_data[idxs]).unsqueeze(-1), torch.tensor(train_data[idxs+1][:, :prediction_length]).unsqueeze(-1)
last_idxs = np.array([i for i in range(0, time_series-1) if not np.isin(i, idxs)])
# Получение тестовых данных


x_test, y_test = torch.tensor(train_data[last_idxs]).unsqueeze(-1), torch.tensor(train_data[last_idxs + 1][:, :prediction_length]).unsqueeze(-1)

torch.Size([800, 20, 1]) torch.Size([800, 10, 1])
(800,)
torch.Size([447, 20, 1])


In [56]:
torch.save(x_train, f'trainX_{SYMBOL}.pt')
torch.save(y_train, f'trainY_{SYMBOL}.pt')
torch.save(x_test, f'testX_{SYMBOL}.pt')
torch.save(y_test, f'testY_{SYMBOL}.pt')